In [1]:
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import MetaTrader5 as mt5
# import pytz module for working with time zone
import pytz, os
import numpy as np

from stockstats import StockDataFrame

# finRL
from finrl.meta.data_processors.processor_alpaca_2 import AlpacaProcessor2
import alpaca_trade_api as tradeapi
from alpaca_trade_api.rest import REST, TimeFrame


# display data on the MetaTrader 5 package
print("MetaTrader5 package author: ",mt5.__author__)
print("MetaTrader5 package version: ",mt5.__version__)

MetaTrader5 package author:  MetaQuotes Ltd.
MetaTrader5 package version:  5.0.37


In [2]:
# establish connection to the MetaTrader 5 terminal
if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
    quit()
 
# display data on MetaTrader 5 version
print(mt5.version())

(500, 3391, '05 Aug 2022')


In [3]:
# MT5 account infor
MT5_ACCOUNT = os.environ.get('MT5_ACCOUNT')
MT5_PASSWORD = os.environ.get('MT5_PASSWORD')
MT5_SERVER = os.environ.get('MT5_SERVER')

# Alpaca
API_KEY = os.environ.get('API_KEY')
API_SECRET = os.environ.get('API_SECRET')
API_BASE_URL = os.environ.get('API_BASE_URL')
data_url = os.environ.get('data_url')
TIME_INTERVAL_RAW = '1Hour'

# Tick info
TICK_START_DATE='2022-07-01'
TICK_END_DATE='2022-07-31'


TIME_INTERVAL = {
    '1Day': mt5.TIMEFRAME_D1,
    '4Hour': mt5.TIMEFRAME_H4,
    '1Hour': mt5.TIMEFRAME_H1,
    '30Min': mt5.TIMEFRAME_M30,
    '15Min': mt5.TIMEFRAME_M15,
    '5Min': mt5.TIMEFRAME_M5,
    '1Min': mt5.TIMEFRAME_M1,
}

tech_indicator_list=[
    "macd",
    "boll_ub",
    "boll_lb",
    "rsi_30",
    "dx_30",
    "close_14_sma", 
    "close_50_sma",
    "close_200_sma",
    "volume_14_sma",
    "volume_xu_volume_14_sma"
]

TIC_LIST = ['EURUSD', 'USDCHF', 'XAUUSD', 'CADCHF']

# MT5 server time: Europe/Moscow  (GMT+3)


## Alpaca

In [4]:
try:
    alpaca_api = tradeapi.REST(API_KEY, API_SECRET, API_BASE_URL, "v2")
except BaseException:
    raise ValueError("Wrong Account Info!")

## MT5

In [5]:
# now connect to another trading account specifying the password
authorized=mt5.login(login=MT5_ACCOUNT, password=MT5_PASSWORD, server=MT5_SERVER)
if authorized:
    # display trading account data 'as is'
    print(mt5.account_info())
    # display trading account data in the form of a list
    print("Show account_info()._asdict():")
    account_info_dict = mt5.account_info()._asdict()
    for prop in account_info_dict:
        print("  {}={}".format(prop, account_info_dict[prop]))
else:
    print("failed to connect at account #{}, error code: {}".format(MT5_ACCOUNT, mt5.last_error()))

AccountInfo(login=6106603, trade_mode=0, leverage=100, limit_orders=0, margin_so_mode=0, trade_allowed=True, trade_expert=True, margin_mode=2, currency_digits=2, fifo_close=False, balance=100000.0, credit=0.0, profit=0.0, equity=100000.0, margin=0.0, margin_free=100000.0, margin_level=0.0, margin_so_call=100.0, margin_so_so=50.0, margin_initial=0.0, margin_maintenance=0.0, assets=0.0, liabilities=0.0, commission_blocked=0.0, name='Dan Da', server='OANDA-OGM MT5 Demo', currency='USD', company='OANDA Corporation')
Show account_info()._asdict():
  login=6106603
  trade_mode=0
  leverage=100
  limit_orders=0
  margin_so_mode=0
  trade_allowed=True
  trade_expert=True
  margin_mode=2
  currency_digits=2
  fifo_close=False
  balance=100000.0
  credit=0.0
  profit=0.0
  equity=100000.0
  margin=0.0
  margin_free=100000.0
  margin_level=0.0
  margin_so_call=100.0
  margin_so_so=50.0
  margin_initial=0.0
  margin_maintenance=0.0
  assets=0.0
  liabilities=0.0
  commission_blocked=0.0
  name=Dan

In [6]:
def fetch_bars(tic_list: [str]) -> pd.DataFrame:
    
    tic_frames = pd.DataFrame()
    
    for tic in tic_list:
        
        # Fetch the bars from MT5
        tic_rates = mt5.copy_rates_range(
                        "EURUSD", 
                        TIME_INTERVAL[TIME_INTERVAL_RAW], 
                        datetime.strptime(TICK_START_DATE, '%Y-%m-%d'), 
                        datetime.strptime(TICK_END_DATE, '%Y-%m-%d')
                    )
        bar_frame = pd.DataFrame(tic_rates) 
        
        # Convert timestamp to datetime
        bar_frame['time'] = pd.to_datetime(bar_frame['time'], unit='s')
        
        # Add tick name
        bar_frame['tic'] = tic
        
        # Rename tick_volume to volume
        bar_frame = bar_frame.rename(columns={"time": "date", "tick_volume": "volume"}, errors="raise")
        
        # Drop column "real_volume"
        bar_frame = bar_frame.drop(columns=['real_volume'])
        
        # Concat bars to the main dataframe
        tic_frames = pd.concat([tic_frames, bar_frame])
        
    return tic_frames

In [7]:
tic_frames = fetch_bars(TIC_LIST)
tic_frames 

,date,open,high,low,close,volume,spread,tic
0,2022-07-01 07:00:00,1.04538,1.04607,1.04466,1.04535,5262,10,EURUSD
1,2022-07-01 08:00:00,1.04534,1.04701,1.04516,1.04626,4986,10,EURUSD
2,2022-07-01 09:00:00,1.04626,1.04697,1.04402,1.04609,8297,10,EURUSD
3,2022-07-01 10:00:00,1.04610,1.04631,1.04322,1.04457,13528,10,EURUSD
4,2022-07-01 11:00:00,1.04458,1.04596,1.04362,1.04584,8372,10,EURUSD
...,...,...,...,...,...,...,...,...
492,2022-07-29 19:00:00,1.02010,1.02103,1.01766,1.01964,6270,10,CADCHF
493,2022-07-29 20:00:00,1.01964,1.02137,1.01932,1.02092,4475,10,CADCHF
494,2022-07-29 21:00:00,1.02092,1.02200,1.02061,1.02194,3513,10,CADCHF
495,2022-07-29 22:00:00,1.02193,1.02224,1.02120,1.02205,3723,10,CADCHF


In [8]:
def add_technical_indicator(
    df: pd.DataFrame,
    tic_list: [str],
    tech_indicator_list=[
        "macd",
        "boll_ub",
        "boll_lb",
        "rsi_30",
        "dx_30",
        "close_14_sma", 
        "close_50_sma",
        "close_200_sma",
        "volume_14_sma",
        "volume_xu_volume_14_sma"
    ],
):
    tic_indicator_frame = pd.DataFrame()

    for tic in tic_list:
        tic_df = df[df['tic'] == tic].copy()
        tic_df = tic_df.sort_values(by=["tic", "date"])

        stock = StockDataFrame.retype(tic_df)
    
        columns = tic_df.columns.values.tolist() + tech_indicator_list
        del columns[0]

        tic_indicator_frame = pd.concat([tic_indicator_frame, stock[columns]])
        tic_indicator_frame.sort_values(by=["date", "tic"])
    
    return tic_indicator_frame.replace(np.nan, 0)

In [9]:
tic_indicator_frame = add_technical_indicator(tic_frames, TIC_LIST)
tic_indicator_frame

,high,low,close,volume,spread,tic,macd,boll_ub,boll_lb,rsi_30,dx_30,close_14_sma,close_50_sma,close_200_sma,volume_14_sma,volume_xu_volume_14_sma
date,,,,,,,,,,,,,,,,
2022-07-01 07:00:00,1.04607,1.04466,1.04535,5262,10,EURUSD,0.000000,0.000000,0.000000,0.000000,0.000000,1.045350,1.045350,1.045350,5262.000000,False
2022-07-01 08:00:00,1.04701,1.04516,1.04626,4986,10,EURUSD,0.000020,1.047092,1.044518,100.000000,100.000000,1.045805,1.045805,1.045805,5124.000000,False
2022-07-01 09:00:00,1.04697,1.04402,1.04609,8297,10,EURUSD,0.000021,1.046868,1.044932,83.804382,12.907348,1.045900,1.045900,1.045900,6181.666667,True
2022-07-01 10:00:00,1.04631,1.04322,1.04457,13528,10,EURUSD,-0.000034,1.047114,1.044021,33.548424,38.817429,1.045567,1.045567,1.045567,8018.250000,False
2022-07-01 11:00:00,1.04596,1.04362,1.04584,8372,10,EURUSD,-0.000013,1.046984,1.044260,56.233699,38.817429,1.045622,1.045622,1.045622,8089.000000,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-29 19:00:00,1.02103,1.01766,1.01964,6270,10,CADCHF,0.000317,1.024291,1.016566,51.080868,2.799333,1.020828,1.018824,1.019503,10580.357143,False
2022-07-29 20:00:00,1.02137,1.01932,1.02092,4475,10,CADCHF,0.000369,1.024342,1.016662,52.621836,0.988869,1.020804,1.019033,1.019487,10537.642857,False
2022-07-29 21:00:00,1.02200,1.02061,1.02194,3513,10,CADCHF,0.000487,1.024485,1.016769,53.820984,2.415266,1.020881,1.019251,1.019464,10449.642857,False


In [10]:
def df_to_array(df: pd.DataFrame, tech_indicator_list: [str]) -> pd.DataFrame:
    df = df.copy()
    unique_ticker = df.tic.unique()
    if_first_time = True

    for tic in unique_ticker:

        if if_first_time:
            
            price_array = df[df.tic == tic][["close"]].values
            tech_array = df[df.tic == tic][tech_indicator_list].values
#             if if_vix:
#                 turbulence_array = df[df.tic == tic]["VIXY"].values
#             else:
#                 turbulence_array = df[df.tic == tic]["turbulence"].values
            if_first_time = False
    
        else:
    
            price_array = np.hstack(
                [price_array, df[df.tic == tic][["close"]].values]
            )
            tech_array = np.hstack(
                [tech_array, df[df.tic == tic][tech_indicator_list].values]
            )
            
    print("Successfully transformed into array")
#     return price_array, tech_array, turbulence_array
    return price_array, tech_array

In [11]:
price_array, tech_array = df_to_array(tic_indicator_frame,tech_indicator_list)

Successfully transformed into array


## Train

In [12]:
from finrl.agents.stablebaselines3.models import DRLAgent as DRLAgent_sb3
from finrl.meta.env_stock_trading.env_stocktrading_np import StockTradingEnv


In [18]:
TOTAL_TIMESTEPS = 1e7

PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.01,
    "learning_rate": 0.00025,
    "batch_size": 128,
}

MODEL_NAME = 'ppo'

env_config = {
        "price_array": price_array,
        "tech_array": tech_array,
#         "turbulence_array": turbulence_array,
        "if_train": True,
    }
env_instance = StockTradingEnv(config=env_config)

# read parameters
cwd = f'./{MODEL_NAME}'

total_timesteps = TOTAL_TIMESTEPS
agent_params = PPO_PARAMS

In [ ]:
agent = DRLAgent_sb3(env=env_instance)
model = agent.get_model(MODEL_NAME, model_kwargs=agent_params)
trained_model = agent.train_model(
    model=model, tb_log_name=MODEL_NAME, total_timesteps=TOTAL_TIMESTEPS
)
print("Training is finished!")
trained_model.save(cwd)
print("Trained model is saved in " + str(cwd))

{'n_steps': 2048, 'ent_coef': 0.01, 'learning_rate': 0.00025, 'batch_size': 128}
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------------
| rollout/           |                |
|    ep_len_mean     | 496            |
|    ep_rew_mean     | -0.0629        |
| time/              |                |
|    fps             | 1946           |
|    iterations      | 1              |
|    time_elapsed    | 1              |
|    total_timesteps | 2048           |
| train/             |                |
|    reward          | -0.00012103932 |
---------------------------------------
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 496           |
|    ep_rew_mean          | -0.0682       |
| time/                   |               |
|    fps                  | 1552          |
|    iterations           | 2             |
|    time_elapsed         | 2        

-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 496           |
|    ep_rew_mean          | -0.0308       |
| time/                   |               |
|    fps                  | 1362          |
|    iterations           | 10            |
|    time_elapsed         | 15            |
|    total_timesteps      | 20480         |
| train/                  |               |
|    approx_kl            | 0.0059594642  |
|    clip_fraction        | 0.0521        |
|    clip_range           | 0.2           |
|    entropy_loss         | -5.69         |
|    explained_variance   | 0.703         |
|    learning_rate        | 0.00025       |
|    loss                 | -0.074        |
|    n_updates            | 90            |
|    policy_gradient_loss | -0.0101       |
|    reward               | 0.00044020085 |
|    std                  | 1             |
|    value_loss           | 0.000873      |
--------------------------------

-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 496           |
|    ep_rew_mean          | -0.0158       |
| time/                   |               |
|    fps                  | 1321          |
|    iterations           | 19            |
|    time_elapsed         | 29            |
|    total_timesteps      | 38912         |
| train/                  |               |
|    approx_kl            | 0.007973596   |
|    clip_fraction        | 0.0533        |
|    clip_range           | 0.2           |
|    entropy_loss         | -5.7          |
|    explained_variance   | 0.397         |
|    learning_rate        | 0.00025       |
|    loss                 | -0.0537       |
|    n_updates            | 180           |
|    policy_gradient_loss | -0.00827      |
|    reward               | -0.0050580315 |
|    std                  | 1.01          |
|    value_loss           | 0.000468      |
--------------------------------

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 496          |
|    ep_rew_mean          | -0.000114    |
| time/                   |              |
|    fps                  | 1313         |
|    iterations           | 28           |
|    time_elapsed         | 43           |
|    total_timesteps      | 57344        |
| train/                  |              |
|    approx_kl            | 0.006538521  |
|    clip_fraction        | 0.0504       |
|    clip_range           | 0.2          |
|    entropy_loss         | -5.73        |
|    explained_variance   | 0.61         |
|    learning_rate        | 0.00025      |
|    loss                 | -0.069       |
|    n_updates            | 270          |
|    policy_gradient_loss | -0.00544     |
|    reward               | 0.0028498613 |
|    std                  | 1.02         |
|    value_loss           | 0.000703     |
------------------------------------------
-----------

-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 496           |
|    ep_rew_mean          | 0.00801       |
| time/                   |               |
|    fps                  | 1314          |
|    iterations           | 37            |
|    time_elapsed         | 57            |
|    total_timesteps      | 75776         |
| train/                  |               |
|    approx_kl            | 0.008986216   |
|    clip_fraction        | 0.0802        |
|    clip_range           | 0.2           |
|    entropy_loss         | -5.75         |
|    explained_variance   | 0.676         |
|    learning_rate        | 0.00025       |
|    loss                 | -0.0782       |
|    n_updates            | 360           |
|    policy_gradient_loss | -0.0105       |
|    reward               | 0.00016700933 |
|    std                  | 1.02          |
|    value_loss           | 0.000735      |
--------------------------------

...

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 496          |
|    ep_rew_mean          | -0.000183    |
| time/                   |              |
|    fps                  | 1304         |
|    iterations           | 1189         |
|    time_elapsed         | 1866         |
|    total_timesteps      | 2435072      |
| train/                  |              |
|    approx_kl            | 0.0069266227 |
|    clip_fraction        | 0.107        |
|    clip_range           | 0.2          |
|    entropy_loss         | -8.48        |
|    explained_variance   | -0.785       |
|    learning_rate        | 0.00025      |
|    loss                 | -0.104       |
|    n_updates            | 11880        |
|    policy_gradient_loss | -0.00765     |
|    reward               | 0.0          |
|    std                  | 2.04         |
|    value_loss           | 2.66e-09     |
------------------------------------------
-----------

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 496         |
|    ep_rew_mean          | -0.000204   |
| time/                   |             |
|    fps                  | 1304        |
|    iterations           | 1198        |
|    time_elapsed         | 1880        |
|    total_timesteps      | 2453504     |
| train/                  |             |
|    approx_kl            | 0.005455163 |
|    clip_fraction        | 0.0561      |
|    clip_range           | 0.2         |
|    entropy_loss         | -8.4        |
|    explained_variance   | -5.79       |
|    learning_rate        | 0.00025     |
|    loss                 | -0.0878     |
|    n_updates            | 11970       |
|    policy_gradient_loss | -0.00347    |
|    reward               | 0.0         |
|    std                  | 2           |
|    value_loss           | 1.65e-07    |
-----------------------------------------
----------------------------------

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 496         |
|    ep_rew_mean          | -0.000198   |
| time/                   |             |
|    fps                  | 1304        |
|    iterations           | 1207        |
|    time_elapsed         | 1894        |
|    total_timesteps      | 2471936     |
| train/                  |             |
|    approx_kl            | 0.008476779 |
|    clip_fraction        | 0.096       |
|    clip_range           | 0.2         |
|    entropy_loss         | -8.38       |
|    explained_variance   | -1.53       |
|    learning_rate        | 0.00025     |
|    loss                 | -0.0813     |
|    n_updates            | 12060       |
|    policy_gradient_loss | -0.0085     |
|    reward               | 0.0         |
|    std                  | 1.99        |
|    value_loss           | 2.64e-09    |
-----------------------------------------
----------------------------------

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 496         |
|    ep_rew_mean          | -0.000156   |
| time/                   |             |
|    fps                  | 1304        |
|    iterations           | 1216        |
|    time_elapsed         | 1908        |
|    total_timesteps      | 2490368     |
| train/                  |             |
|    approx_kl            | 0.006167774 |
|    clip_fraction        | 0.0614      |
|    clip_range           | 0.2         |
|    entropy_loss         | -8.36       |
|    explained_variance   | -7.38       |
|    learning_rate        | 0.00025     |
|    loss                 | -0.0591     |
|    n_updates            | 12150       |
|    policy_gradient_loss | -0.00651    |
|    reward               | 0.0         |
|    std                  | 1.98        |
|    value_loss           | 9.82e-10    |
-----------------------------------------
----------------------------------

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 496          |
|    ep_rew_mean          | -0.000108    |
| time/                   |              |
|    fps                  | 1304         |
|    iterations           | 1225         |
|    time_elapsed         | 1922         |
|    total_timesteps      | 2508800      |
| train/                  |              |
|    approx_kl            | 0.0067417957 |
|    clip_fraction        | 0.0649       |
|    clip_range           | 0.2          |
|    entropy_loss         | -8.32        |
|    explained_variance   | -0.485       |
|    learning_rate        | 0.00025      |
|    loss                 | -0.0766      |
|    n_updates            | 12240        |
|    policy_gradient_loss | -0.00506     |
|    reward               | 0.0          |
|    std                  | 1.96         |
|    value_loss           | 1.07e-07     |
------------------------------------------
-----------

## Test

In [17]:
episode_total_assets = DRLAgent_sb3.DRL_prediction_load_from_file(
            model_name=MODEL_NAME, environment=env_instance, cwd=cwd
        )
episode_total_assets

Successfully load model ./ppo
episode_return 1.0001644133678897
Test Finished!


[965028.3689775498,
 965028.3556542695,
 965028.3546338379,
 965028.326711591,
 965028.2651551926,
 965028.142829058,
 965027.9414006742,
 965027.4363317998,
 965026.2363782712,
 965025.8255936621,
 965025.1899878014,
 965026.1254144695,
 965025.4147178939,
 965026.1928114116,
 965027.5765435797,
 965028.0104529471,
 965027.541707917,
 965026.5072501184,
 965029.1488869225,
 965028.232016041,
 965028.4382527815,
 965028.2430430314,
 965027.8095131558,
 965026.589333093,
 965026.8399131971,
 965027.9576930598,
 965026.0574344539,
 965027.2130140213,
 965028.8862827223,
 965029.9583337378,
 965032.3354355359,
 965032.3151163727,
 965031.9091963558,
 965030.3275442271,
 965023.5763750463,
 965024.7497072368,
 965023.2944622902,
 965022.574779108,
 965023.6401294379,
 965023.1571676163,
 965022.6986217527,
 965021.891261808,
 965024.2291057633,
 965026.1873544701,
 965026.4890146764,
 965026.8894407097,
 965024.305784138,
 965026.5094343801,
 965027.6894757731,
 965029.6975105947,
 965026.